In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import nfp

print(f"tensorflow {tf.__version__}")
print(f"nfp {nfp.__version__}")

tensorflow 2.3.1
nfp 0.2.0+0.g72982c1.dirty


In [2]:
# Load the input data, here YSI (10.1016/j.combustflame.2017.12.005)
ysi = pd.read_csv('../data/ysi.csv')
ysi.head()

,Species,CAS,Ref,Type,YSI,YSI_err,SMILES
0,"1,2-diphenylbenzene",84-15-1,2.0,aromatic,1338.9,50.0,c1ccc(-c2ccccc2-c2ccccc2)cc1
1,cyclopenta[def]phenanthrene,203-64-5,2.0,aromatic,1312.8,73.5,c1cc2c3c(c1)ccc1cccc(c13)C2
2,fluoranthene,206-44-0,2.0,aromatic,1291.9,72.4,c1ccc2c(c1)-c1cccc3cccc-2c13
3,"1,3-diphenylbenzene",92-06-8,2.0,aromatic,1286.6,72.3,c1ccc(-c2cccc(-c3ccccc3)c2)cc1
4,pyrene,129-00-0,2.0,aromatic,1250.1,70.1,c1cc2ccc3cccc4ccc(c1)c2c34


In [3]:
# Split the data into training, validation, and test sets
valid, test, train = np.split(ysi.SMILES.sample(frac=1., random_state=1), [50, 100])
len(train), len(valid), len(test)

(341, 50, 50)

In [4]:
# Define how to featurize the input molecules

def atom_featurizer(atom):
    """ Return an string representing the atom type
    """

    return str((
        atom.GetSymbol(),
        atom.GetIsAromatic(),
        nfp.get_ring_size(atom, max_size=6),
        atom.GetDegree(),
        atom.GetTotalNumHs(includeNeighbors=True)
    ))


def bond_featurizer(bond, flipped=False):
    """ Get a similar classification of the bond type.
    Flipped indicates which 'direction' the bond edge is pointing. """
    
    if not flipped:
        atoms = "{}-{}".format(
            *tuple((bond.GetBeginAtom().GetSymbol(),
                    bond.GetEndAtom().GetSymbol())))
    else:
        atoms = "{}-{}".format(
            *tuple((bond.GetEndAtom().GetSymbol(),
                    bond.GetBeginAtom().GetSymbol())))
    
    btype = str(bond.GetBondType())
    ring = 'R{}'.format(nfp.get_ring_size(bond, max_size=6)) if bond.IsInRing() else ''
    
    return " ".join([atoms, btype, ring]).strip()


preprocessor = nfp.SmilesPreprocessor(atom_features=atom_featurizer, bond_features=bond_featurizer,
                                      explicit_hs=False)

In [5]:
# Initially, the preprocessor has no data on atom types, so we have to loop over the 
# training set once to pre-allocate these mappings
print("before pre-allocating")
print(preprocessor.atom_tokenizer._data)

for smiles in train:
    preprocessor(smiles, train=True)
    
print()
print("after pre-allocating")
print(preprocessor.atom_tokenizer._data)

before pre-allocating
{'unk': 1}

after pre-allocating
{'unk': 1, "('C', False, 0, 1, 3)": 2, "('C', False, 0, 2, 2)": 3, "('C', False, 0, 3, 0)": 4, "('O', False, 0, 1, 0)": 5, "('O', False, 0, 2, 0)": 6, "('C', False, 0, 1, 2)": 7, "('C', False, 0, 2, 1)": 8, "('C', False, 0, 3, 1)": 9, "('C', False, 0, 4, 0)": 10, "('C', True, 'max', 2, 1)": 11, "('C', True, 'max', 3, 0)": 12, "('C', False, 'max', 2, 2)": 13, "('O', False, 0, 1, 1)": 14, "('C', False, 0, 1, 1)": 15, "('C', False, 0, 2, 0)": 16, "('C', False, 'max', 3, 1)": 17, "('N', False, 0, 1, 0)": 18, "('Br', False, 0, 1, 0)": 19, "('C', False, 'max', 2, 1)": 20, "('C', True, 5, 3, 0)": 21, "('C', False, 5, 2, 2)": 22, "('C', False, 5, 2, 1)": 23, "('C', False, 5, 3, 1)": 24, "('C', False, 5, 3, 0)": 25, "('C', False, 'max', 3, 0)": 26, "('O', False, 5, 2, 0)": 27, "('O', False, 'max', 2, 0)": 28, "('F', False, 0, 1, 0)": 29, "('C', True, 5, 2, 1)": 30, "('C', False, 'max', 4, 0)": 31, "('Cl', False, 0, 1, 0)": 32, "('C', False,

In [6]:
# Main input types for a SMILES-based prediction
smiles = 'CCO'

# Atom types, as integer classes
preprocessor(smiles, train=True)['atom']

array([ 2,  3, 14])

In [7]:
# Bond types, as integer classes
preprocessor(smiles, train=True)['bond']

array([2, 2, 5, 6])

In [8]:
# A connectivity array, where row i indicates bond i connects atom j to atom k
preprocessor(smiles, train=True)['connectivity']

array([[0, 1],
       [1, 0],
       [1, 2],
       [2, 1]])

In [9]:
# Construct the tf.data pipeline. There's a lot of specifying data types and
# expected shapes for tensorflow to pre-allocate the necessary arrays. But 
# essentially, this is responsible for calling the input constructor, batching 
# together multiple molecules, and padding the resulting molecules so that all
# molecules in the same batch have the same number of atoms (we pad with zeros,
# hence why the atom and bond types above start with 1 as the unknown class)

train_dataset = tf.data.Dataset.from_generator(
    lambda: ((preprocessor(row.SMILES, train=False), row.YSI)
             for i, row in ysi[ysi.SMILES.isin(train)].iterrows()),
    output_types=(preprocessor.output_types, tf.float32),
    output_shapes=(preprocessor.output_shapes, []))\
    .cache().shuffle(buffer_size=200)\
    .padded_batch(batch_size=64, 
                  padded_shapes=(preprocessor.padded_shapes(), []),
                  padding_values=(preprocessor.padding_values, 0.))\
    .prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(
    lambda: ((preprocessor(row.SMILES, train=False), row.YSI)
             for i, row in ysi[ysi.SMILES.isin(valid)].iterrows()),
    output_types=(preprocessor.output_types, tf.float32),
    output_shapes=(preprocessor.output_shapes, []))\
    .cache()\
    .padded_batch(batch_size=64, 
                  padded_shapes=(preprocessor.padded_shapes(), []),
                  padding_values=(preprocessor.padding_values, 0.))\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [10]:
## Define the keras model
from tensorflow.keras import layers

# Input layers
atom = layers.Input(shape=[None], dtype=tf.int64, name='atom')
bond = layers.Input(shape=[None], dtype=tf.int64, name='bond')
connectivity = layers.Input(shape=[None, 2], dtype=tf.int64, name='connectivity')

num_features = 8  # Controls the size of the model

# Convert from a single integer defining the atom state to a vector
# of weights associated with that class
atom_state = layers.Embedding(preprocessor.atom_classes, num_features,
                              name='atom_embedding', mask_zero=True)(atom)

# Ditto with the bond state
bond_state = layers.Embedding(preprocessor.bond_classes, num_features,
                              name='bond_embedding', mask_zero=True)(bond)

# Here we use our first nfp layer. This is an attention layer that looks at
# the atom and bond states and reduces them to a single, graph-level vector. 
# mum_heads * units has to be the same dimension as the atom / bond dimension
global_state = nfp.GlobalUpdate(units=8, num_heads=1)([atom_state, bond_state, connectivity])

for _ in range(3):  # Do the message passing
    new_bond_state = nfp.EdgeUpdate()([atom_state, bond_state, connectivity, global_state])
    bond_state = layers.Add()([bond_state, new_bond_state])
    
    new_atom_state = nfp.NodeUpdate()([atom_state, bond_state, connectivity, global_state])
    atom_state = layers.Add()([atom_state, new_atom_state])
    
    new_global_state = nfp.GlobalUpdate(units=8, num_heads=1)(
        [atom_state, bond_state, connectivity, global_state]) 
    global_state = layers.Add()([global_state, new_global_state])

    
# Since the final prediction is a single, molecule-level property (YSI), we 
# reduce the last global state to a single prediction.
ysi_prediction = layers.Dense(1)(global_state)

# Construct the tf.keras model
model = tf.keras.Model([atom, bond, connectivity], [ysi_prediction])

In [11]:
model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1E-3))

# Fit the model. The first epoch is slower, since it needs to cache
# the preprocessed molecule inputs
model.fit(train_dataset, validation_data=valid_dataset, epochs=25)

Epoch 1/25


/Users/pstjohn/miniconda3/envs/nfp/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['n_atom', 'n_bond', 'bond_indices'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


6/6 [==============================] - 1s 201ms/step - loss: 192.5282 - val_loss: 156.3338
Epoch 2/25
6/6 [==============================] - 0s 10ms/step - loss: 191.6269 - val_loss: 154.2021
Epoch 3/25
6/6 [==============================] - 0s 11ms/step - loss: 187.9288 - val_loss: 147.4886
Epoch 4/25
6/6 [==============================] - 0s 11ms/step - loss: 178.0559 - val_loss: 131.6612
Epoch 5/25
6/6 [==============================] - 0s 11ms/step - loss: 162.7336 - val_loss: 124.0303
Epoch 6/25
6/6 [==============================] - 0s 11ms/step - loss: 162.0391 - val_loss: 125.7313
Epoch 7/25
6/6 [==============================] - 0s 10ms/step - loss: 154.8945 - val_loss: 116.7943
Epoch 8/25
6/6 [==============================] - 0s 10ms/step - loss: 149.2845 - val_loss: 113.8111
Epoch 9/25
6/6 [==============================] - 0s 11ms/step - loss: 143.0998 - val_loss: 105.6354
Epoch 10/25
6/6 [==============================] - 0s 10ms/step - loss: 127.5234 - val_loss: 93.4604


In [12]:
# Here, we create a test dataset that doesn't assume we know the values for the YSI

test_dataset = tf.data.Dataset.from_generator(
    lambda: (preprocessor(smiles, train=False)
             for smiles in test),
    output_types=preprocessor.output_types,
    output_shapes=preprocessor.output_shapes)\
    .padded_batch(batch_size=64, 
                  padded_shapes=preprocessor.padded_shapes(),
                  padding_values=preprocessor.padding_values)\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [13]:
# Here are the predictions on the test set
test_predictions = model.predict(test_dataset)
test_db_values = ysi.set_index('SMILES').reindex(test).YSI.values

np.abs(test_db_values - test_predictions.flatten()).mean()

40.5671657333374